In [1]:
!pip install transformers

  Using cached transformers-4.33.3-py3-none-any.whl (7.6 MB)
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Using cached safetensors-0.3.3-cp310-cp310-win_amd64.whl (266 kB)
  Using cached tokenizers-0.13.3-cp310-cp310-win_amd64.whl (3.5 MB)
  Using cached PyYAML-6.0.1-cp310-cp310-win_amd64.whl (145 kB)
     ---------------------------------------- 15.8/15.8 MB 1.5 MB/s eta 0:00:00
  Using cached regex-2023.8.8-cp310-cp310-win_amd64.whl (268 kB)
  Using cached huggingface_hub-0.17.3-py3-none-any.whl (295 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
  Using cached fsspec-2023.9.2-py3-none-any.whl (173 kB)
  Using cached certifi-2023.7.22-py3-none-any.whl (158 kB)
     -------------------------------------- 123.8/123.8 kB 2.4 MB/s eta 0:00:00
  Using cached charset_normalizer-3.2.0-cp310-cp310-win_amd64.whl (96 kB)
  Using cached idna-3.4-py3-none-any.whl (61 kB)



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from bs4 import BeautifulSoup
import requests

C:\Users\manav\anaconda3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
!pip install sentencepiece

  Using cached sentencepiece-0.1.99-cp310-cp310-win_amd64.whl (977 kB)



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip install tensorflow

In [12]:
!pip3 install torch torchvision torchaudio

In [14]:
import torch

# 1.Setup Summarising Model 

In [5]:
model_name="human-centered-summarization/financial-summarization-pegasus"
tokenizer=PegasusTokenizer.from_pretrained(model_name)
model=PegasusForConditionalGeneration.from_pretrained(model_name)






















































































































































































































































































































































































































































C:\Users\manav\anaconda3\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\manav\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To

# 2.Summarise an article 

In [6]:
url="https://au.finance.yahoo.com/news/bank-hit-major-outage-043800328.html"
r = requests.get(url)
soup=BeautifulSoup(r.text,'html.parser')
paragraphs=soup.find_all("p")

In [7]:
paragraphs[0].text

'A bank outage that left customers unable to pay for everyday goods before the start of the holiday weekend has largely been resolved.'

In [8]:
text=[paragraph.text for paragraph in paragraphs]
words=' '.join(text).split(' ')[:400]

In [9]:
ARTICLE=' '.join(words)

In [10]:
ARTICLE

'A bank outage that left customers unable to pay for everyday goods before the start of the holiday weekend has largely been resolved. ANZ customers took to social media on Friday amid reports users were unable to access the bank’s online service since about midday. Customers also reported being unable to reach the bank via their telephone line, with one person stating that “everyone is having problems”. A spokesperson for the bank confirmed “most systems” were back online by about 5pm, including internet banking, ATMs, and the ANZ app. “However, we are still experiencing some instability across our Visa Credit Card transaction services.” the spokesperson said. “We recommend customers use Debit Card or Cash wherever accepted while our teams are continuing to work to resolve.” The spokesperson said the widespread outage was caused by a software issue which impacted a number of our “core systems” from 1.37pm. The issue resulted in multiple internal and customer facing programs to shutter

In [11]:
input_ids=tokenizer.encode(ARTICLE,return_tensors='pt')
output=model.generate(input_ids,max_length=55,num_beams=5,early_stopping=True)
summary=tokenizer.decode(output[0],skip_special_tokens=True)

In [12]:
summary

'Customers express frustration on social media over outage. ANZ says ‘most systems’ back online by 5pm'

# 4.Build a News and Sentiment Pipeline 

In [18]:
#companies: Game Stop, Tesla, Bitcoin
monitored_tickers=['GME','TSLA','BTC']

## Search for Stock News using Google and Yahoo Finance 

In [16]:
def search_for_stock_news_urls(ticker):
    search_url=f"https://www.google.com/search?q=yahoo+finance+{ticker}&sca_esv=569660528&tbm=nws"
    r=requests.get(search_url)
    soup=BeautifulSoup(r.text,"html.parser")
    atags=soup.find_all('a')
    hrefs=[link["href"] for link in atags]
    return hrefs

In [17]:
raw_urls={ticker:search_for_stock_news_urls(ticker) for ticker in monitored_tickers}
raw_urls

{'GME': ['/?sa=X&ved=0ahUKEwjX8u2H5dGBAxWoGbkGHfoaDSEQOwgC',
  '/search?q=yahoo+finance+GME&tbm=nws&sca_esv=569660528&ie=UTF-8&gbv=1&sei=PssXZdfbAqiz5OUP-rW0iAI',
  '/search?q=yahoo+finance+GME&sca_esv=569660528&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwjX8u2H5dGBAxWoGbkGHfoaDSEQ_AUIBSgA',
  '/search?q=yahoo+finance+GME&sca_esv=569660528&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwjX8u2H5dGBAxWoGbkGHfoaDSEQ_AUIBygC',
  '/search?q=yahoo+finance+GME&sca_esv=569660528&ie=UTF-8&tbm=bks&source=lnms&sa=X&ved=0ahUKEwjX8u2H5dGBAxWoGbkGHfoaDSEQ_AUICCgD',
  '/search?q=yahoo+finance+GME&sca_esv=569660528&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwjX8u2H5dGBAxWoGbkGHfoaDSEQ_AUICSgE',
  'https://maps.google.com/maps?q=yahoo+finance+GME&sca_esv=569660528&um=1&ie=UTF-8&sa=X&ved=0ahUKEwjX8u2H5dGBAxWoGbkGHfoaDSEQ_AUICigF',
  '/search?q=yahoo+finance+GME&sca_esv=569660528&ie=UTF-8&tbm=shop&source=lnms&sa=X&ved=0ahUKEwjX8u2H5dGBAxWoGbkGHfoaDSEQ_AUICygG',
  '/advanced_search',
  '/search?q=yahoo+finance+GM

In [20]:
raw_urls["TSLA"][0]

'/?sa=X&ved=0ahUKEwjfn7KI5dGBAxWtFLkGHWR2C0MQOwgC'

## Strip out unwanted URLs

In [35]:
import re

In [36]:
exclude_list=['maps','policies','preferences','accounts','support']

In [47]:
def strip_unwanted_urls(urls,exclude_list):
    val=[]
    for url in urls:
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res=re.findall(r'(https?://\S+)',url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

In [48]:
cleaned_urls={ticker:strip_unwanted_urls(raw_urls[ticker],exclude_list) for ticker in monitored_tickers}

In [49]:
cleaned_urls

{'GME': ['https://www.barrons.com/articles/gamestop-stock-price-ryan-cohen-aa31dd8b',
  'https://www.marketwatch.com/story/gamestop-stock-soars-after-ryan-cohen-named-ceo-b37f71be',
  'https://qz.com/emails/daily-brief/1850885687/memeing-to-the-top',
  'https://www.cnbc.com/2023/09/28/gamestop-names-ryan-cohen-as-ceo-effective-immediately.html',
  'https://www.forbes.com/sites/maryroeloffs/2023/09/28/billionaire-chair-ryan-cohen-named-gamestop-ceo-years-after-meme-stock-surge/',
  'https://seekingalpha.com/news/4016093-biggest-stock-movers-today-peloton-micron-technology-and-more',
  'https://www.google.com/search?q%3Dyahoo%2Bfinance%2BGME%26sca_esv%3D569660528%26tbm%3Dnws%26pccc%3D1',
  'https://www.daijiworld.com/news/newsDisplay%3FnewsID%3D1125375',
  'https://www.moneycontrol.com/news/trends/gamestop-ceo-ryan-cohen-will-get-zero-salary-his-email-to-staff-11449581.html',
  'https://www.thestreet.com/memestocks/gme/gamestop-stock-danger-ahead-for-short-sellers',
  'https://www.news18

## Search and Scrape Cleaned Urls 

In [63]:
def scrape_and_search(urls):
    ARTICLES=[]
    for url in urls:
        r=requests.get(url)
        soup=BeautifulSoup(r.text,'html.parser')
        paragraphs=soup.find_all('p')
        text=[paragraph.text for paragraph in paragraphs]
        words=' '.join(text).split(' ')[:300]
        ARTICLE=" ".join(words)
        ARTICLES.append(ARTICLE)
    return ARTICLES

In [64]:
articles={ticker:scrape_and_search(cleaned_urls[ticker]) for ticker in monitored_tickers}
articles

{'GME': ["That content doesn't seem to exist... have you tried going to the home page, or exploring an interest? Meanwhile, here are some important items you'll want to know:",
  "Shares of GameStop Corp. surged Thursday, after the consumer electronics retailer named Ryan Cohen as its president and chief executive officer, effective immediately. The move, which wasn’t unexpected, comes a few months after the company fired then-CEO Matthew Furlong, and elected activist investor Ryan Cohen as executive chairman. At that time, Cohen said in a tweet, “Not for long.” Cohen will relinquish his role as executive chairman when he is also appointed chairman. GameStop’s stock GME shot up 7% in premarket trading, putting it on track to open at a three-week high. Read: Ryan Cohen becomes GameStop CEO and social media reacts: ‘Changing the paradigm on Wall Street.’ Cohen, who previously founded and was CEO Chewy Inc. \n        CHWY,\n        -0.05%,\n       is the manager of RC Ventures, which in J

## Summarize all articles 

In [65]:
def summarize(articles):
    summaries=[]
    for article in articles:
        input_ids=tokenizer.encode(article,return_tensors='pt')
        output=model.generate(input_ids,max_length=55,num_beams=5,early_stopping=True)
        summary=tokenizer.decode(output[0],skip_special_tokens=True)
        summaries.append(summary)
    return summaries

In [66]:
summaries={ticker:summarize(articles[ticker]) for ticker in monitored_tickers}
summaries

{'GME': ['Have you tried going to Newsround?',
  'Cohen elected as executive chairman of the company in June. Activist investor previously founded and was CEO Chewy',
  'A judge upholds minimum wage for New York City delivery workers. Tesla sued for racial harassment, but won’t comment',
  'Best Credit Cards for You Best Rewards Credit Cards Best 0% APR Credit Cards Best Balance Transfer Credit Cards',
  'Chewy founder to serve as CEO, but won’t be compensated. Video game retailer has had a string of CEO changes since 2021',
  'Jabil, Peloton, Workday, Micron slip on mixed results.',
  'All images are copyrighted.',
  'Mangalore is set to get a new airport.',
  'Ryan Cohen to take over as CEO, chairman of the video game retailer.',
  '.',
  'Ryan Cohen appointed as CEO and Chairman of the company.'],
 'TSLA': ['Best Credit Cards for You Best Rewards Credit Cards Best 0% APR Credit Cards Best Balance Transfer Credit Cards',
  'Candidates can apply for jobs at Tesla, Tesla AI, or any oth

In [67]:
summaries["BTC"]

['Yield-bearing Bitcoin Minetrix to be biggest winner after raising $200,000.',
 'CoinDesk DeFi Index leads crypto rally with 5.5% gain. LDO, ARB, AAVE, MKR among top gainers',
 'The U.S. dollar index reached a 10-month high on Thursday. Ether also made up some ground after hitting a seven-day high on Wednesday',
 'Bitcoin-Ether implied volatility spread has been negative for 20 days',
 'Largest crypto by market capitalization is up 3.2% in September. Ethereum to go live next week as futures-based ETF planned',
 'ordinary shares and warrants will begin trading on Nasdaq under the ticker symbols "BTCT" and "BTCTW".',
 'All images are copyrighted.',
 'Revenues were lower due to weakness in cryptocurrency markets. Average dollar value of BTC mined in 2023 was $23,740.44',
 'We are aware of the issue and are working to resolve it.',
 'New crypto presale offers no-fuss, no-rigs, low-cost stake-to-mine token.',
 'Regulator delays decision on ARK 21Shares Bitcoin ETF application.']

# 5.Adding Sentiment Analysis 

In [68]:
from transformers import pipeline
sentiment=pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [69]:
sentiment(summaries["BTC"])

[{'label': 'POSITIVE', 'score': 0.994099497795105},
 {'label': 'POSITIVE', 'score': 0.9606256484985352},
 {'label': 'POSITIVE', 'score': 0.9810195565223694},
 {'label': 'NEGATIVE', 'score': 0.9996230602264404},
 {'label': 'NEGATIVE', 'score': 0.9403278231620789},
 {'label': 'NEGATIVE', 'score': 0.9920476078987122},
 {'label': 'NEGATIVE', 'score': 0.9880996346473694},
 {'label': 'NEGATIVE', 'score': 0.998678982257843},
 {'label': 'POSITIVE', 'score': 0.9979088306427002},
 {'label': 'NEGATIVE', 'score': 0.9977009892463684},
 {'label': 'NEGATIVE', 'score': 0.9968666434288025}]

In [70]:
scores={ticker:sentiment(summaries[ticker]) for ticker in monitored_tickers}
scores

{'GME': [{'label': 'NEGATIVE', 'score': 0.9915708303451538},
  {'label': 'POSITIVE', 'score': 0.9967880249023438},
  {'label': 'NEGATIVE', 'score': 0.6145312190055847},
  {'label': 'POSITIVE', 'score': 0.998146653175354},
  {'label': 'NEGATIVE', 'score': 0.6854579448699951},
  {'label': 'NEGATIVE', 'score': 0.9995985627174377},
  {'label': 'NEGATIVE', 'score': 0.9880996346473694},
  {'label': 'NEGATIVE', 'score': 0.9141563177108765},
  {'label': 'POSITIVE', 'score': 0.996704638004303},
  {'label': 'POSITIVE', 'score': 0.9668781757354736},
  {'label': 'POSITIVE', 'score': 0.9984606504440308}],
 'TSLA': [{'label': 'POSITIVE', 'score': 0.998146653175354},
  {'label': 'NEGATIVE', 'score': 0.729343056678772},
  {'label': 'POSITIVE', 'score': 0.9515891075134277},
  {'label': 'NEGATIVE', 'score': 0.9896793365478516},
  {'label': 'NEGATIVE', 'score': 0.8962414860725403},
  {'label': 'NEGATIVE', 'score': 0.9994315505027771},
  {'label': 'NEGATIVE', 'score': 0.9880996346473694},
  {'label': 'NEG

In [71]:
print(summaries["GME"][3],scores["GME"][3]["label"],scores["GME"][3]["score"])

Best Credit Cards for You Best Rewards Credit Cards Best 0% APR Credit Cards Best Balance Transfer Credit Cards POSITIVE 0.998146653175354


In [72]:
scores["BTC"][0]["score"]

0.994099497795105

# 6.Exporting Results to CSV

In [74]:
summaries

{'GME': ['Have you tried going to Newsround?',
  'Cohen elected as executive chairman of the company in June. Activist investor previously founded and was CEO Chewy',
  'A judge upholds minimum wage for New York City delivery workers. Tesla sued for racial harassment, but won’t comment',
  'Best Credit Cards for You Best Rewards Credit Cards Best 0% APR Credit Cards Best Balance Transfer Credit Cards',
  'Chewy founder to serve as CEO, but won’t be compensated. Video game retailer has had a string of CEO changes since 2021',
  'Jabil, Peloton, Workday, Micron slip on mixed results.',
  'All images are copyrighted.',
  'Mangalore is set to get a new airport.',
  'Ryan Cohen to take over as CEO, chairman of the video game retailer.',
  '.',
  'Ryan Cohen appointed as CEO and Chairman of the company.'],
 'TSLA': ['Best Credit Cards for You Best Rewards Credit Cards Best 0% APR Credit Cards Best Balance Transfer Credit Cards',
  'Candidates can apply for jobs at Tesla, Tesla AI, or any oth

In [75]:
scores

{'GME': [{'label': 'NEGATIVE', 'score': 0.9915708303451538},
  {'label': 'POSITIVE', 'score': 0.9967880249023438},
  {'label': 'NEGATIVE', 'score': 0.6145312190055847},
  {'label': 'POSITIVE', 'score': 0.998146653175354},
  {'label': 'NEGATIVE', 'score': 0.6854579448699951},
  {'label': 'NEGATIVE', 'score': 0.9995985627174377},
  {'label': 'NEGATIVE', 'score': 0.9880996346473694},
  {'label': 'NEGATIVE', 'score': 0.9141563177108765},
  {'label': 'POSITIVE', 'score': 0.996704638004303},
  {'label': 'POSITIVE', 'score': 0.9668781757354736},
  {'label': 'POSITIVE', 'score': 0.9984606504440308}],
 'TSLA': [{'label': 'POSITIVE', 'score': 0.998146653175354},
  {'label': 'NEGATIVE', 'score': 0.729343056678772},
  {'label': 'POSITIVE', 'score': 0.9515891075134277},
  {'label': 'NEGATIVE', 'score': 0.9896793365478516},
  {'label': 'NEGATIVE', 'score': 0.8962414860725403},
  {'label': 'NEGATIVE', 'score': 0.9994315505027771},
  {'label': 'NEGATIVE', 'score': 0.9880996346473694},
  {'label': 'NEG

In [76]:
cleaned_urls

{'GME': ['https://www.barrons.com/articles/gamestop-stock-price-ryan-cohen-aa31dd8b',
  'https://www.marketwatch.com/story/gamestop-stock-soars-after-ryan-cohen-named-ceo-b37f71be',
  'https://qz.com/emails/daily-brief/1850885687/memeing-to-the-top',
  'https://www.cnbc.com/2023/09/28/gamestop-names-ryan-cohen-as-ceo-effective-immediately.html',
  'https://www.forbes.com/sites/maryroeloffs/2023/09/28/billionaire-chair-ryan-cohen-named-gamestop-ceo-years-after-meme-stock-surge/',
  'https://seekingalpha.com/news/4016093-biggest-stock-movers-today-peloton-micron-technology-and-more',
  'https://www.google.com/search?q%3Dyahoo%2Bfinance%2BGME%26sca_esv%3D569660528%26tbm%3Dnws%26pccc%3D1',
  'https://www.daijiworld.com/news/newsDisplay%3FnewsID%3D1125375',
  'https://www.moneycontrol.com/news/trends/gamestop-ceo-ryan-cohen-will-get-zero-salary-his-email-to-staff-11449581.html',
  'https://www.thestreet.com/memestocks/gme/gamestop-stock-danger-ahead-for-short-sellers',
  'https://www.news18

In [78]:
range(len(summaries['GME']))

range(0, 11)

In [96]:
summaries['GME'][10]

'Ryan Cohen appointed as CEO and Chairman of the company.'

In [99]:
def create_output_array(summaries,scores,urls):
    output=[]
    for ticker in monitored_tickers:
        for counter in range(len(summaries[ticker])):
            output_this=[
                ticker,
                summaries[ticker][counter],
                scores[ticker][counter]["label"],
                scores[ticker][counter]["score"],
                urls[ticker][counter]
            ]
            output.append(output_this)
    return output

In [100]:
final_output=create_output_array(summaries,scores,cleaned_urls)
final_output

[['GME',
  'Have you tried going to Newsround?',
  'NEGATIVE',
  0.9915708303451538,
  'https://www.barrons.com/articles/gamestop-stock-price-ryan-cohen-aa31dd8b'],
 ['GME',
  'Cohen elected as executive chairman of the company in June. Activist investor previously founded and was CEO Chewy',
  'POSITIVE',
  0.9967880249023438,
  'https://www.marketwatch.com/story/gamestop-stock-soars-after-ryan-cohen-named-ceo-b37f71be'],
 ['GME',
  'A judge upholds minimum wage for New York City delivery workers. Tesla sued for racial harassment, but won’t comment',
  'NEGATIVE',
  0.6145312190055847,
  'https://qz.com/emails/daily-brief/1850885687/memeing-to-the-top'],
 ['GME',
  'Best Credit Cards for You Best Rewards Credit Cards Best 0% APR Credit Cards Best Balance Transfer Credit Cards',
  'POSITIVE',
  0.998146653175354,
  'https://www.cnbc.com/2023/09/28/gamestop-names-ryan-cohen-as-ceo-effective-immediately.html'],
 ['GME',
  'Chewy founder to serve as CEO, but won’t be compensated. Video ga

In [101]:
final_output.insert(0,['Ticker','Summary','Label','Confidence','URL'])

In [102]:
final_output

[['Ticker', 'Summary', 'Label', 'Confidence', 'URL'],
 ['GME',
  'Have you tried going to Newsround?',
  'NEGATIVE',
  0.9915708303451538,
  'https://www.barrons.com/articles/gamestop-stock-price-ryan-cohen-aa31dd8b'],
 ['GME',
  'Cohen elected as executive chairman of the company in June. Activist investor previously founded and was CEO Chewy',
  'POSITIVE',
  0.9967880249023438,
  'https://www.marketwatch.com/story/gamestop-stock-soars-after-ryan-cohen-named-ceo-b37f71be'],
 ['GME',
  'A judge upholds minimum wage for New York City delivery workers. Tesla sued for racial harassment, but won’t comment',
  'NEGATIVE',
  0.6145312190055847,
  'https://qz.com/emails/daily-brief/1850885687/memeing-to-the-top'],
 ['GME',
  'Best Credit Cards for You Best Rewards Credit Cards Best 0% APR Credit Cards Best Balance Transfer Credit Cards',
  'POSITIVE',
  0.998146653175354,
  'https://www.cnbc.com/2023/09/28/gamestop-names-ryan-cohen-as-ceo-effective-immediately.html'],
 ['GME',
  'Chewy found

In [104]:
import csv
with open('assetsummaries.csv','w',newline='') as f:
    csv_writer=csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerows(final_output)